# Modèle LDA

In [35]:
import pandas as pd
import re
import gensim
import nltk
from gensim.parsing.preprocessing import preprocess_string, STOPWORDS
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from itertools import combinations, permutations
# from google.colab import drive

In [36]:
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /Users/perrine/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/perrine/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /Users/perrine/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [37]:
# drive.mount('/content/drive')

In [38]:
dataset = "Wikipedia"

# Charger le dataset
if dataset == "Stackoverflow":
  dataframe_lda = pd.read_json("../data/stackoverflow-data-idf.json", lines=True)
elif dataset == "Wikipedia":
  dataframe_lda = pd.read_json("../data/wikipedia_keywords.json")


# Afficher le schéma
print("Schema:\n")
print(dataframe_lda.dtypes, "\n")
print("Number of questions, columns=", dataframe_lda.shape)

Schema:

keywords    object
sentence    object
dtype: object 

Number of questions, columns= (13014, 2)


In [39]:
# Division du jeu de données en 60% train, 20% dev, 20% test
train_df, temp_df = train_test_split(dataframe_lda, test_size=0.4, random_state=42)
dev_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Afficher les dimensions des ensembles pour vérifier
print("Train set:", train_df.shape)
print("Dev set:", dev_df.shape)
print("Test set:", test_df.shape)

Train set: (7808, 2)
Dev set: (2603, 2)
Test set: (2603, 2)


## Prétraitement

In [40]:
stemmer = SnowballStemmer('english')

def lemmatize_stemming(text:str) -> str:
    """
    Lemmatize and stem the text

    Parameters:
    text : str : text to be lemmatized and stemmed

    Returns:
    str : lemmatized and stemmed text
    """
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='n'))


def pre_process(text:str) -> str:
    """
    Convert to lowercase, remove html tags, remove special characters and numbers

    Parameters:
    text : str : text to be cleaned

    Returns:
    text : str : cleaned text
    """
    text = text.lower()
    text = re.sub("</?.*?>"," <> ",text)
    text=re.sub("(\\d|\\W)+"," ",text)

    return text

def get_stop_words(stop_file_path: str) -> frozenset:
    """
    Load stop words

    Parameters:
    stop_file_path : str : path to the stop words file

    Returns:
    stop_set : set : set of stop words
    """
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)

stopwords = get_stop_words("../data/stopwords_json.txt")

def stopWords(text: str) -> str:
    """
    Remove stopwords and words with less than 3 characters

    Parameters:
    text : str : text to be cleaned

    Returns:
    result : str : cleaned text
    """
    result = ""
    for token in gensim.utils.simple_preprocess(text):
        if token not in STOPWORDS:
            if token not in stopwords:
                result += lemmatize_stemming(token) + " "
    return result

# Appliquer la fonction de nettoyage au texte
for df in [train_df, dev_df, test_df]:
  if dataset == "Stackoverflow":
      df['text'] = df['title'] + " " + df['body']
  else:
      df['text'] = df['sentence']
  df['text'] = df['text'].apply(pre_process).apply(stopWords)


# Affichage des résultats du nettoyage
print("Exemple de texte nettoyé :\n")
print(df['text'][:5])

Exemple de texte nettoyé :

10408    laurenc interview sweeney crew awar refer airp...
4099     histor russian athlet success contend olymp game 
5768     italian defeat prompt germani deploy expeditio...
8974         genotyp evolut model hardi weinberg principl 
5350     februari churchil commiss second lieuten th qu...
Name: text, dtype: object


Ce bloc de code est un processus de prétraitement de texte pour le traitement du langage naturel. Il vise à préparer le texte pour des analyses ultérieures en utilisant des techniques de lemmatisation, de stemming, et un filtrage avancé des stopwords. Voici une explication simplifiée de chaque étape :

1. **Initialisation d'un stemmer** :
   - Un objet `stemmer` de type `SnowballStemmer` est créé pour l'anglais. Cette étape permet de réduire les mots à leur racine, en éliminant les suffixes qui ne sont pas nécessaires pour la compréhension du sens général du mot.

2. **Fonction `lemmatize_stemming`** :
   - Cette fonction prend un mot en entrée, le transforme en sa forme de base ou lemmatise en utilisant `WordNetLemmatizer`, ciblant spécifiquement les noms (`pos='n'`). Ensuite, elle applique le stemming pour simplifier encore plus le mot. Cette combinaison aide à normaliser les mots pour une analyse cohérente.

3. **Nettoyage du texte avec `pre_process`** :
   - Convertit tout le texte en minuscules pour uniformiser la casse.
   - Supprime les balises HTML et les caractères spéciaux ou numériques, nettoyant le texte de tout élément qui pourrait perturber l'analyse.

4. **Chargement et application des stopwords** :
   - La fonction `get_stop_words` charge une liste personnalisée de mots fréquemment utilisés mais peu informatifs (stopwords) à partir d'un fichier. Ces mots sont souvent exclus de l'analyse pour se concentrer sur les termes plus significatifs.

5. **Filtrage et prétraitement des mots** :
   - La fonction `stopWords` tokenise le texte, exclut les mots trop courts ou les stopwords, et applique les opérations de lemmatisation et de stemming aux mots restants. Le résultat est un texte nettoyé et réduit à des termes essentiels.

6. **Application du prétraitement aux données** :
   - Selon le dataset (`Stackoverflow` ou autre), les titres et les corps des textes sont combinés en une seule colonne `text`.
   - Les fonctions de nettoyage et de prétraitement sont appliquées successivement pour transformer le texte brut en une forme optimisée pour l'analyse.

7. **Affichage des résultats** :
   - Les premiers exemples de texte prétraité sont affichés, montrant l'efficacité des étapes de nettoyage et de normalisation dans la préparation des données pour des analyses plus poussées, comme la modélisation de sujets.

Ces étapes garantissent que le texte est suffisamment simplifié et uniformisé, ce qui est crucial pour obtenir de bons résultats dans des applications de NLP telles que l'analyse de sentiments ou la modélisation de sujets.

In [41]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_words([word for text in train_df['text'] for word in text.split()])
finder.apply_freq_filter(10)
bigram_score = finder.score_ngrams(bigram_measures.pmi)


bigrams = {}
for bigram, score in bigram_score:
    bigram = '_'.join(bigram)
    bigrams[bigram] = score

bigrams

{'brest_litovsk': 14.476417130701066,
 'sgt_pepper': 14.33891360695113,
 'alsac_lorrain': 14.21338272486727,
 'ba_maw': 14.21338272486727,
 'href_wiki': 14.201410083201194,
 'puerto_rico': 14.097905507447335,
 'warner_bros': 13.990990303530822,
 'ed_sullivan': 13.202494408724535,
 'hong_kong': 13.097905507447335,
 'benioff_wei': 13.043457723424957,
 'kievan_ru': 12.948038158346275,
 'saint_petersburg': 12.940364230460855,
 'harmon_oscil': 12.862885477783136,
 'meter_ft': 12.710882384338086,
 'kaiser_wilhelm': 12.45042192216812,
 'los_angel': 12.364844367737044,
 'dive_bomber': 12.350886248617206,
 'nobel_prize': 12.268524279059731,
 'herbert_hoover': 12.263423407366865,
 'sq_mi': 12.243756373910788,
 'mu_nu': 12.016985512063766,
 'austro_hungarian': 11.859444235077287,
 'vacuum_tube': 11.83700684595753,
 'alan_ture': 11.789356442361171,
 'labrador_retriev': 11.717971809124405,
 'benito_mussolini': 11.669062208643458,
 'midget_submarin': 11.60570014764603,
 'kennel_club': 11.52280266950

In [42]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = nltk.collocations.TrigramCollocationFinder.from_words([word for text in train_df['text'] for word in text.split()])
finder.apply_freq_filter(25)
trigram_score = finder.score_ngrams(trigram_measures.pmi)

# print("\nTrigram score:\n")
# trigram_score[:10]

trigrams = {}
for trigram, score in trigram_score:
    trigram = '_'.join(trigram)
    trigrams[trigram] = score

trigrams

{'attack_pearl_harbor': 19.570729049915386,
 'new_york_citi': 16.348064845025952,
 'late_th_centuri': 15.386564929432556,
 'earli_th_centuri': 14.888386194853464,
 'world_war_ii': 14.789785183455376,
 'new_york_time': 14.667954073522704,
 'second_world_war': 11.671997805348237,
 'war_unit_state': 9.559794886823731}

## Entrainement

In [43]:
# modèle LDA
def train_lda(data, num_topics=10, passes=200) -> tuple:
    """
    Train LDA model on the data

    Parameters:
    data : DataFrame : data to train the model on
    num_topics : int : number of topics to train the model on
    passes : int : number of passes for training the model

    Returns:
    tuple : (lda_model, dictionary, bow_corpus)
        lda_model : LdaModel : trained LDA model
        dictionary : Dictionary : Gensim dictionary
        bow_corpus : list : bag of words corpus
    """
    processed_docs = [preprocess_string(doc) for doc in data['text']]
    dictionary = Dictionary(processed_docs)
    dictionary.filter_extremes(no_below=20, no_above=0.85, keep_n=1000)
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
    lda_model = LdaModel(bow_corpus, num_topics=num_topics, id2word=dictionary, passes=passes, random_state=42)
    return lda_model, dictionary, bow_corpus, processed_docs


def evaluate_model(model, texts: list, dictionary) -> float:
    """
    Evaluate the LDA model using Coherence Score

    Parameters:
    model : LdaModel : trained LDA model
    texts : list : list of texts
    dictionary : Dictionary : Gensim dictionary

    Returns:
    float : coherence score of the model
    """
    coherence_model = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
    return coherence_model.get_coherence()


data_df = train_df
print("Training LDA model on data...")
lda_model, dictionary, bow_corpus, processed_docs = train_lda(data_df, 30, 250)
print("Coherence Score on Training Data:", evaluate_model(lda_model, processed_docs, dictionary))


Training LDA model on data...
Coherence Score on Training Data: 0.5345474529003926


Ce bloc de code illustre la préparation, l'entraînement et l'évaluation d'un modèle de Latent Dirichlet Allocation (LDA) avec la bibliothèque Gensim, destiné à la détection de thèmes dans des documents textuels. Voici une explication simplifiée de chaque partie du code :

1. **Préparation et entraînement du modèle LDA** :
   - **Fonction `train_lda`** : Cette fonction configure et entraîne un modèle LDA. Elle prend en entrée les documents textuels, le nombre de sujets désirés (`num_topics`), et le nombre de passages sur le corpus (`passes`) pour l'optimisation.
   - **Dictionnaire et corpus** : Les documents sont d'abord transformés en une liste de tokens (`processed_docs`), puis en un dictionnaire (`dictionary`) qui assigne un identifiant unique à chaque mot. Le corpus est converti en format sac-de-mots (`bow_corpus`), où chaque document est représenté par une liste de paires (identifiant de mot, fréquence).
   - **Paramètres du modèle** : Le modèle est paramétré pour extraire un certain nombre de sujets (`num_topics`), avec une association explicite entre les mots et leurs identifiants (`id2word=dictionary`), et est optimisé à travers plusieurs passes sur les données.

2. **Évaluation de la cohérence du modèle** :
   - **Fonction `evaluate_model`** : Cette fonction utilise `CoherenceModel` pour évaluer la cohérence du modèle LDA, un indicateur de la qualité des thèmes extraits. Une cohérence élevée signifie que les mots fréquents d'un sujet apparaissent souvent ensemble dans les documents, suggérant que le sujet est significatif et cohérent.
   - **Arguments** : Le modèle LDA (`model`), les documents prétraités (`texts`), et le dictionnaire sont nécessaires pour calculer la cohérence, qui se base sur la fréquence de co-occurrence des mots.

3. **Exécution et résultats** :
   - **Entraînement et évaluation** : Le modèle est entraîné et évalué sur le `train_df` après l'extraction et la préparation des documents. La cohérence des thèmes générés est ensuite affichée, fournissant un feedback sur la pertinence des thèmes identifiés.

4. **Implications pratiques** :
   - **Temps de calcul** : L'entraînement d'un modèle LDA, en particulier avec un grand nombre de passes et de sujets, peut être long (indiqué par "47 minutes"), et dépend fortement de la taille du corpus et des ressources système disponibles.
   - **Ajustement des paramètres** : Le score de cohérence aide à ajuster les paramètres du modèle pour améliorer la pertinence des sujets extraits, en fonction des objectifs de l'analyse.

Ce processus est crucial pour l'analyse de grands ensembles de textes où la détection de thèmes sous-jacents est nécessaire pour la compréhension ou la classification des documents.

In [44]:
# Extraire les mots-clé pour chaque document
def extract_keywords_for_document(lda_model, bow_corpus: list, documents: list, num_keywords: int) -> list:
    """
    Extract keywords for each document

    Parameters:
    lda_model : LdaModel : trained LDA model
    bow_corpus : list : bag of words corpus
    documents : list : list of documents
    num_keywords : int : number of keywords to extract

    Returns:
    list : list of dictionaries containing keywords and their scores for each document
    """
    document_keywords_scores = []
    for bow_doc, doc_text in zip(bow_corpus, documents):
        doc_topics = lda_model.get_document_topics(bow_doc, minimum_probability=0.01)
        doc_topics = sorted(doc_topics, key=lambda x: x[1], reverse=True)
        keywords_scores = {}
        for topic_num, topic_weight in doc_topics:
            topic_words = lda_model.show_topic(topic_num, topn=100)
            for word, word_weight in topic_words:
                if word in doc_text:
                    if word not in keywords_scores:
                        keywords_scores[word] = word_weight * topic_weight
                    else:
                        keywords_scores[word] += word_weight * topic_weight

        keywords_scores_sorted = dict(sorted(keywords_scores.items(), key=lambda item: item[1], reverse=True))
        top_keywords_scores = dict(list(keywords_scores_sorted.items())[:num_keywords])
        document_keywords_scores.append(top_keywords_scores)

    return document_keywords_scores


# Extraire les mots-clés et leurs scores avec la fonction modifiée
keywords_and_scores = extract_keywords_for_document(lda_model, bow_corpus, processed_docs, num_keywords=10)
print(keywords_and_scores[:5])


[{'frame': 0.0145837795, 'event': 0.0054012015, 'clear': 0.0017843783, 'local': 0.0016530053, 'addit': 0.0007509848}, {'record': 0.027270636, 'relea': 0.019521128, 'album': 0.01541515, 'beatl': 0.013352306, 'singl': 0.011891185, 'hollywood': 0.011616089, 'studio': 0.009913859, 'includ': 0.009185504, 'tour': 0.005398599, 'origin': 0.004929646}, {'churchil': 0.028603062, 'parti': 0.017305389, 'conserv': 0.007528526, 'gener': 0.0074543105, 'labour': 0.0035542364}, {'nation': 0.037965335, 'product': 0.022768166, 'increa': 0.020536266, 'govern': 0.015615228}, {'british': 0.030386414, 'forc': 0.018510345, 'occupi': 0.009239134, 'empir': 0.00803592, 'march': 0.0072682453, 'attack': 0.006636343, 'line': 0.0064065317, 'april': 0.005773251, 'earli': 0.0047080503, 'follow': 0.003633895}]


Ce bloc de code permet d'extraire les mots-clés les plus pertinents pour chaque document d'un corpus, en utilisant un modèle de Latent Dirichlet Allocation (LDA) déjà entraîné. Voici une explication simple et structurée de chaque étape de ce processus :

1. **Définition de la fonction d'extraction** :
   - **`extract_keywords_for_document`** : Cette fonction est configurée pour prendre en entrée un modèle LDA, un corpus en format sac de mots (`bow_corpus`), les documents sous forme textuelle, et le nombre désiré de mots-clés à extraire (`num_keywords`).

2. **Extraction de mots-clés par document** :
   - **Récupération des distributions des sujets** : Pour chaque document, le modèle LDA est consulté pour obtenir une distribution des sujets (avec un seuil de probabilité minimale) qui indique la contribution relative de chaque sujet au document.
   - **Tri des sujets** : Les sujets sont triés par leur poids (importance) pour prioriser les sujets les plus influents pour le document en question.

3. **Sélection et pondération des mots-clés** :
   - **Extraction des termes par sujet** : Pour chaque sujet pertinent, les termes les plus significatifs sont extraits à l'aide de la méthode `show_topic`, qui renvoie les mots et leur poids dans ce sujet particulier.
   - **Calcul du score des mots-clés** : Le score de chaque mot-clé est ajusté en fonction de son poids dans le sujet et le poids de ce sujet dans le document. Ce calcul assure que les mots-clés sélectionnés sont non seulement centraux pour les sujets mais aussi spécifiquement pertinents pour le document.

4. **Filtrage et classement des mots-clés** :
   - **Match avec le texte du document** : Seuls les mots qui figurent effectivement dans le document sont conservés. Cela garantit la pertinence contextuelle des mots-clés.
   - **Classement des mots-clés** : Les mots-clés sont ensuite triés par leur score total, et les `num_keywords` les plus pertinents sont retenus pour chaque document.

5. **Résultats et vérifications** :
   - **Compilation des résultats** : La liste des mots-clés pour chaque document est stockée dans une liste de dictionnaires, facilitant l'analyse ou l'affichage ultérieur.
   - **Affichage des premiers résultats** : Pour vérification, les mots-clés des premiers documents sont imprimés, montrant ainsi les termes les plus influents et leur applicabilité aux textes analysés.

Cette méthode offre une manière systématique et quantifiable de déterminer les termes clés qui caractérisent le contenu de documents divers, en se basant sur une analyse statistique des thèmes via LDA. Elle est essentielle pour des tâches comme la résumé automatique, la catégorisation de contenu ou encore l'analyse de tendances dans de grands ensembles de données textuelles.

In [45]:
# Verify if the combination of keywords and permutation is present in bigrams and trigrams dictionary. If present, add the bigram/trigam to the keyword
def verification_bigram_trigram(keyword: dict, bigrams: dict, trigrams: dict) -> dict:
    """
    Verify if the combination of keywords and permutation is present in bigrams and trigrams dictionary. If present, add the bigram/trigam to the keyword

    Parameters:
    keyword : dict : dictionary containing keywords and their scores
    bigrams : dict : dictionary containing bigrams and their scores
    trigrams : dict : dictionary containing trigrams and their scores

    Returns:
    dict : updated dictionary containing keywords and their scores
    """
    keywords = list(keyword.keys())
    bigrams = set(bigrams.keys())
    trigrams = set(trigrams.keys())

    combined_keywords = set()
    for k in range(1, min(4, len(keywords) + 1)):  # Use min to ensure we do not exceed 3 tokens
        for subset in combinations(keywords, k):
            for perm in permutations(subset):
                combined_keyword = '_'.join(perm)
                if len(combined_keyword.split()) <= 3:  # Check the token count
                    combined_keywords.add(combined_keyword)

    for combined_keyword in combined_keywords:
        if combined_keyword in bigrams or combined_keyword in trigrams:
            keyword[combined_keyword] = 0.001  # Add a small value to indicate it is a bigram/trigram

    return keyword

# Vérifier si la combinaison de mots-clés et la permutation sont présentes dans le dictionnaire des bigrams et trigrams
for i in range(len(keywords_and_scores)):
    keywords_and_scores[i] = verification_bigram_trigram(keywords_and_scores[i], bigrams, trigrams)

print(keywords_and_scores[:10])


[{'frame': 0.0145837795, 'event': 0.0054012015, 'clear': 0.0017843783, 'local': 0.0016530053, 'addit': 0.0007509848}, {'record': 0.027270636, 'relea': 0.019521128, 'album': 0.01541515, 'beatl': 0.013352306, 'singl': 0.011891185, 'hollywood': 0.011616089, 'studio': 0.009913859, 'includ': 0.009185504, 'tour': 0.005398599, 'origin': 0.004929646}, {'churchil': 0.028603062, 'parti': 0.017305389, 'conserv': 0.007528526, 'gener': 0.0074543105, 'labour': 0.0035542364, 'conserv_parti': 0.001}, {'nation': 0.037965335, 'product': 0.022768166, 'increa': 0.020536266, 'govern': 0.015615228, 'nation_govern': 0.001}, {'british': 0.030386414, 'forc': 0.018510345, 'occupi': 0.009239134, 'empir': 0.00803592, 'march': 0.0072682453, 'attack': 0.006636343, 'line': 0.0064065317, 'april': 0.005773251, 'earli': 0.0047080503, 'follow': 0.003633895, 'british_empir': 0.001, 'british_forc': 0.001}, {'hungarian': 0.0069324034, 'territori': 0.00663382, 'romania': 0.0050703166}, {'roosevelt': 0.023476351, 'presid': 0

In [46]:
# Récupérer les colonnes
def get_text_columns(dataset: str, dataframe) -> list:
  """
    Get the text columns and tags from the dataframe

    Parameters:
    dataset : str : dataset name
    dataframe : DataFrame : dataframe to get the columns from

    Returns:
    if dataset == "Stackoverflow":
        list: list of titles, bodys and tags
    elif dataset == "Wikipedia":
        list: list of sentences and keywords
  """
  if dataset == "Stackoverflow":
    titles = dataframe['title'].tolist()
    bodys = dataframe['body'].tolist()
    tags = [ ' '.join([lemmatize_stemming(word) for word in word_tokenize(tag.replace('|', ' ').lower())]) for tag in dataframe['tags'].tolist()]
    return titles, bodys, tags
  elif dataset == "Wikipedia":
    sentences = dataframe['sentence'].tolist()
    tags = [[lemmatize_stemming(keyword.lower()) for keyword in keyword_list] for keyword_list in dataframe['keywords']]
    return sentences, tags

def calculate_precision_recall_fscore(true_tags: list, predicted_keywords: dict) -> tuple:
    """
    Calculate precision, recall and f-score for the predicted keywords

    Parameters:
    true_tags : list : list of true tags
    predicted_keywords : dict : dictionary of predicted keywords

    Returns:
    tuple : (precision, recall, f_score)
        precision : float : precision score
        recall : float : recall score
        f_score : float : f-score
    """
    true_set = set(true_tags)
    predicted_set = set(predicted_keywords.keys())  # Les clés sont les mots-clés
    tp = len(true_set & predicted_set)  # True positives
    fp = len(predicted_set - true_set)  # False positives
    fn = len(true_set - predicted_set)  # False negatives

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    if precision + recall > 0:
        f_score = 2 * precision * recall / (precision + recall)
    else:
        f_score = 0

    return precision, recall, f_score

def matches_keywords(keyword_scores: list, tags: list) -> list:
    """
    Check if the keywords match the tags.

    Parameters:
    keyword_scores : list : list of dictionaries containing keywords and their scores
    tags : list : list of tags

    Returns:
    list : list of numbers indicating the count of keyword matches per tag list
    """
    pourcentages:list=[]
    matches = []
    for tag_list, keyword_dict in zip(tags, keyword_scores):
        keywords = list(keyword_dict.keys())
        # print('keywords: ', keywords, '\ntag_list: ', tag_list)
        tag_ngram = []
        # S'il y a des tags de bigrams ou trigrams séparés par espace, on le remplace par '_'
        for tag in tag_list:
            tag = tag.strip().replace(' ', '_')
            # print(tag)
            tag_ngram.append(tag)

        match = 0
        bingo_pourcentage:str = ""
        for keyword in keywords:
            if keyword in tag_ngram:
                match += 1
        bingo_pourcentage = f"{round(match / len(tag_ngram), 3) * 100}%"
        pourcentages.append(bingo_pourcentage)
        matches.append(match)
    return matches, pourcentages


if dataset == "Stackoverflow":
  titles, bodys, tags = get_text_columns(dataset, data_df)
  tags = [tag.split(' ') for tag in tags]
  evaluation_scores = [calculate_precision_recall_fscore(tags[i], keywords_and_scores[i]) for i in range(len(tags))]
  match_keywords, match_percent = matches_keywords(keywords_and_scores, tags)
  dataframe_lda = pd.DataFrame(zip(titles, bodys, tags, keywords_and_scores, match_keywords, match_percent), columns=['title', 'body', 'tags', 'keywords', 'match', 'match percent'])
  # Add scores to DataFrame
  dataframe_lda['precision'], dataframe_lda['recall'], dataframe_lda['f_score'] = zip(*evaluation_scores)
  dataframe_lda.to_csv("../results/csv/Stackoverflow_lda-keywords-match-train_df.csv", index=False)

  # Affichage des résultats
  print(dataframe_lda[['title', 'tags', 'keywords', 'match', 'match percent', 'precision', 'recall', 'f_score']].head())

elif dataset == "Wikipedia":
  sentences, tags = get_text_columns(dataset, data_df)
  evaluation_scores = [calculate_precision_recall_fscore(tags[i], keywords_and_scores[i]) for i in range(len(tags))]
  match_keywords, match_percent = matches_keywords(keywords_and_scores, tags)
  dataframe_lda = pd.DataFrame(zip(sentences, tags, keywords_and_scores, match_keywords, match_percent), columns=['sentences', 'tags', 'keywords', 'match', 'match percent'])

  # Add scores to DataFrame
  dataframe_lda['precision'], dataframe_lda['recall'], dataframe_lda['f_score'] = zip(*evaluation_scores)
  dataframe_lda.to_csv("../results/csv/Wikipedia_lda-keywords-match-train_df.csv", index=False)

  # Affichage des résultats
  print(dataframe_lda[['sentences', 'tags', 'keywords', 'match', 'match percent','precision', 'recall', 'f_score']].head())

                                           sentences  \
0  It is clear that the two events that are simul...   
1  When the Beatles' studio albums were released ...   
2  In May, Churchill was still generally unpopula...   
3  Government spending increased from 8.0% of the...   
4  In early 1918, the front line was extended and...   

                                                tags  \
0                                            [event]   
1  [past mast, magical mystery tour, singl, apple...   
2                                     [labour parti]   
3                                       [under hoov]   
4  [extend, jordan valley, first transjordan, sec...   

                                            keywords  match match percent  \
0  {'frame': 0.0145837795, 'event': 0.0054012015,...      1        100.0%   
1  {'record': 0.027270636, 'relea': 0.019521128, ...      1         20.0%   
2  {'churchil': 0.028603062, 'parti': 0.017305389...      0          0.0%   
3  {'nation': 0.03

Ce bloc de code réalise l'extraction et le traitement des colonnes textuelles à partir de DataFrames spécifiques, évalue l'exactitude des mots-clés extraits en utilisant les métriques de précision, rappel, et F-score, puis enregistre et affiche les résultats. Voici une explication simplifiée de chaque partie :

1. **Extraction des données textuelles** :
   - **`get_text_columns`** : Cette fonction adapte son comportement en fonction du dataset spécifié (`"Stackoverflow"` ou `"Wikipedia"`). Pour "Stackoverflow", elle récupère les titres, corps de texte, et les tags transformés pour l'analyse. Pour "Wikipedia", elle extrait les phrases et traite les mots-clés associés.

2. **Calcul des métriques d'évaluation** :
   - **`calculate_precision_recall_fscore`** : Cette fonction calcule la précision, le rappel, et le score F (F-score) pour évaluer l'efficacité avec laquelle les mots-clés prédits correspondent aux tags réels. Elle utilise des ensembles pour déterminer les vrais positifs, faux positifs, et faux négatifs entre les tags et les mots-clés prédits.

3. **Traitement des correspondances de mots-clés** :
   - **`matches_keywords`** : Cette fonction compte le nombre de correspondances entre les tags réels et les combinaison mots-clés prédits allant jusqu'à la combinaison de 3 tokens, offrant une mesure directe de la pertinence des mots-clés extraits par rapport aux tags attendus.

4. **Création et manipulation de DataFrames** :
   - En fonction du dataset, les données extraites et les scores calculés sont organisés dans des DataFrames. Ces DataFrames incluent des colonnes pour les textes, les tags, les mots-clés prédits, le nombre de correspondances, et les scores de précision, rappel, et F-score.

5. **Enregistrement et affichage des résultats** :
   - Les DataFrames finaux sont enregistrés en format CSV pour une utilisation future ou une analyse plus approfondie.
   - Les premières entrées du DataFrame sont également affichées pour donner un aperçu immédiat des résultats de l'analyse.

Ce processus permet une évaluation systématique de l'efficacité des techniques d'extraction de mots-clés, adaptée à des contextes variés tels que les questions techniques de "Stackoverflow" ou les articles encyclopédiques de "Wikipedia". Les résultats obtenus peuvent aider à affiner les méthodes d'extraction de mots-clés ou à valider l'exactitude des modèles utilisés.

## Test

In [47]:
# Préparation des données de test
test_docs = [preprocess_string(doc) for doc in test_df['text']]
test_bow_corpus = [dictionary.doc2bow(doc) for doc in test_docs]

# Extraction des mots-clés pour les documents de test
test_keywords_and_scores = extract_keywords_for_document(lda_model, test_bow_corpus, test_docs, num_keywords=10)

# Vérification des bigrams et trigrams dans les mots-clés de test
for i in range(len(test_keywords_and_scores)):
    test_keywords_and_scores[i] = verification_bigram_trigram(test_keywords_and_scores[i], bigrams, trigrams)

# Extraction des colonnes de texte et des tags pour le dataset de test
if dataset == "Stackoverflow":
    test_titles, test_bodys, test_tags = get_text_columns(dataset, test_df)
    test_tags = [tag.split(' ') for tag in test_tags]
    test_evaluation_scores = [calculate_precision_recall_fscore(test_tags[i], test_keywords_and_scores[i]) for i in range(len(test_tags))]
    test_match_keywords, test_match_percent = matches_keywords(test_keywords_and_scores, test_tags)
    test_dataframe_lda = pd.DataFrame(zip(test_titles, test_bodys, test_tags, test_keywords_and_scores, test_match_keywords, test_match_percent), columns=['title', 'body', 'tags', 'keywords', 'match', 'match percent'])
    test_dataframe_lda['precision'], test_dataframe_lda['recall'], test_dataframe_lda['f_score'] = zip(*test_evaluation_scores)
    test_dataframe_lda.to_csv("../results/csv/Stackoverflow_lda-keywords-match-test_df.csv", index=False)

    print(test_dataframe_lda[['title', 'tags', 'keywords', 'match', 'match percent', 'precision', 'recall', 'f_score']].head())

elif dataset == "Wikipedia":
    test_sentences, test_tags = get_text_columns(dataset, test_df)
    test_evaluation_scores = [calculate_precision_recall_fscore(test_tags[i], test_keywords_and_scores[i]) for i in range(len(test_tags))]
    test_match_keywords, test_match_percent = matches_keywords(test_keywords_and_scores, test_tags)
    test_dataframe_lda = pd.DataFrame(zip(test_sentences, test_tags, test_keywords_and_scores, test_match_keywords, test_match_percent), columns=['sentences', 'tags', 'keywords', 'match', 'match percent'])
    test_dataframe_lda['precision'], test_dataframe_lda['recall'], test_dataframe_lda['f_score'] = zip(*test_evaluation_scores)
    test_dataframe_lda.to_csv("../results/csv/Wikipedia_lda-keywords-match-test_df.csv", index=False)

    print(test_dataframe_lda[['sentences', 'tags', 'keywords', 'match', 'match percent', 'precision', 'recall', 'f_score']].head())

                                           sentences  \
0  Laurence had interviewed Sweeney and his crew,...   
1  Historically, Russian athletes have been one o...   
2  Italian defeats prompted Germany to deploy an ...   
3  Genotype evolution can be modeled with the Har...   
4  In February 1895, Churchill was commissioned a...   

                                                tags  \
0                                 [the great artist]   
1             [russian athlet, olympic gam, russian]   
2  [deploy an expeditionary forc, rommel, afrika ...   
3                          [hardy-weinberg principl]   
4  [4th queen's own hussar, british armi, aldershot]   

                                            keywords  match  \
0  {'great': 0.012375866, 'refer': 0.0044525727, ...      0   
1  {'game': 0.017060347, 'success': 0.011271413, ...      1   
2  {'forc': 0.017408913, 'germani': 0.016419487, ...      1   
3   {'model': 0.0072758105, 'principl': 0.007131724}      0   
4  {'church